In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, StructType

In [0]:
spark

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "YOUR_CLIENT_ID",
    "fs.azure.account.oauth2.client.secret": "YOUR_SECRET",
    "fs.azure.account.oauth2.client.endpoint": "YOUR_ENDPOINT"
    
}

## Mounting Data Storage Container with App Storage

In [0]:
dbutils.fs.mount(
source = "AZURE_DATALAKE_LOCATION", # contrainer@storageacc
mount_point = "/mnt/data-storage-container",
extra_configs = configs)

In [0]:
%fs
ls "/mnt/data-storage-container"

path,name,size,modificationTime
dbfs:/mnt/data-storage-container/CopiedFolder/,CopiedFolder/,0,1729882789000
dbfs:/mnt/data-storage-container/FileTest/,FileTest/,0,1729882436000
dbfs:/mnt/data-storage-container/MovedFolder/,MovedFolder/,0,1729882982000
dbfs:/mnt/data-storage-container/NYCTripSmall.parquet,NYCTripSmall.parquet,1267079,1730368752000
dbfs:/mnt/data-storage-container/Unemployment.csv,Unemployment.csv,210955,1729878029000
dbfs:/mnt/data-storage-container/datasets/,datasets/,0,1729878728000
dbfs:/mnt/data-storage-container/kafka-checkpoints/,kafka-checkpoints/,0,1735141960000
dbfs:/mnt/data-storage-container/output/,output/,0,1730377369000
dbfs:/mnt/data-storage-container/stream_output/,stream_output/,0,1734998685000
dbfs:/mnt/data-storage-container/tonystark.txt,tonystark.txt,449,1729785705000


In [0]:
schema = StructType().add("key", StringType()).add("value", StringType())

In [0]:
cosmos_db_uri = 'YOUR_MONGODB_URI' 

In [0]:
print(cosmos_db_uri)

In [0]:
# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaToMongoDB") \
    .getOrCreate()



In [0]:
# Define your Kafka credentials
sasl_username = "DW4BDBUFSNYEWTE2"  # Replace with your API key
sasl_password = "yFxHp0S787aiuElHry8KxfOrKUwgFPe/hJUFopDJFgzdzKu/w/hX/3zmecD1+YxQ"  # Replace with your Secret key

In [0]:
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "pkc-921jm.us-east-2.aws.confluent.cloud:9092") \
    .option("subscribe", "news-topic") \
    .option("kafka.sasl.jaas.config", 
            f"org.apache.kafka.common.security.plain.PlainLoginModule required username='{sasl_username}' password='{sasl_password}';") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("startingOffsets", "earliest") \
    .load()


In [0]:
kafka_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [0]:
messages = kafka_stream.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [0]:
query = messages.writeStream \
    .format("mongodb") \
    .option("spark.mongodb.write.connection.uri", cosmos_db_uri) \
    .option("spark.mongodb.write.database", "my_db") \
    .option("spark.mongodb.write.collection", "coll_2") \
    .outputMode("append") \
    .option("checkpointLocation", "/mnt/data-storage-container/kafka-checkpoints/") \
    .start()

query.awaitTermination()